In [17]:
import os
import csv
from pymongo import MongoClient

# --- Configuration ---
MONGO_HOST_TEMPLATE = "mongodb://mongodb-replicaset.{}.svc.cluster.local:27017"
MONGO_DB_NAME = "domino"
MONGO_COLLECTION = "projects"
OUTPUT_FILE = "projects_with_git_repos.csv"

# --- Get credentials and platform namespace ---
platform_namespace = os.environ.get("DOMINO_PLATFORM_NAMESPACE", "domino-platform")
mongo_uri = MONGO_HOST_TEMPLATE.format(platform_namespace)

client = MongoClient(
    mongo_uri,
    username="admin",
    password="fL9SUiN0ClYzEfKEi1MolvXyM3IaMxoE",
    authSource="admin",
    authMechanism="SCRAM-SHA-1",
)

db = client[MONGO_DB_NAME]
collection = db[MONGO_COLLECTION]

# --- Define aggregation pipeline ---
pipeline = [
    {
        "$match": {
            "isArchived": { "$ne": True },
            "mainRepository.uri": { "$exists": True, "$ne": None }
        }
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "ownerId",
            "foreignField": "_id",
            "as": "owner"
        }
    },
    { "$unwind": "$owner" },
    {
        "$project": {
            "_id": 0,
            "user": "$owner.fullName",
            "email": "$owner.email",
            "projectName": "$name",
            "mainRepository": "$mainRepository.uri",
            "importedRepositories": {
                "$map": {
                    "input": { "$ifNull": ["$repositories", []] },
                    "as": "repo",
                    "in": "$$repo.uri"
                }
            }
        }
    }
]

# --- Run aggregation and write to CSV ---
with open(OUTPUT_FILE, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = None
    for doc in collection.aggregate(pipeline, allowDiskUse=True, batchSize=500):
        doc["importedRepositories"] = ", ".join(doc.get("importedRepositories", []))
        
        if writer is None:
            writer = csv.DictWriter(csvfile, fieldnames=doc.keys())
            writer.writeheader()
        
        writer.writerow(doc)

print(f"Done. Exported to {OUTPUT_FILE}")


Done. Exported to projects_with_git_repos.csv


In [2]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.5 MB/s eta 0:00:00


In [7]:
user = db.users.find_one({ "email": "test-notifs+integration-test@dominodatalab.com" })
print(user["_id"])


6841f10349b4c497b102e30b


In [13]:
import os
from pymongo import MongoClient
from bson import ObjectId

# --- Config ---
MONGO_HOST_TEMPLATE = "mongodb://mongodb-replicaset.{}.svc.cluster.local:27017"
MONGO_DB_NAME = "domino"
MONGO_COLLECTION = "projects"
OLD_DOMAIN = "https://github.com"
NEW_DOMAIN = "https://github.enterprise.com"

TEST_USER_ID = ObjectId("6841f10349b4c497b102e30b")  # Example user
TEST_PROJECT_NAME = "WineQualityProject"  # Project to test with

# --- Connection ---
platform_namespace = os.environ.get("DOMINO_PLATFORM_NAMESPACE", "domino-platform")
mongo_uri = MONGO_HOST_TEMPLATE.format(platform_namespace)

client = MongoClient(
    mongo_uri,
    username="admin",
    password="fL9SUiN0ClYzEfKEi1MolvXyM3IaMxoE",
    authSource="admin",
    authMechanism="SCRAM-SHA-1",
)

db = client[MONGO_DB_NAME]
projects = db[MONGO_COLLECTION]

# --- Helper to swap domains ---
def replace_domain(uri: str) -> str:
    if uri and uri.startswith(OLD_DOMAIN):
        return uri.replace(OLD_DOMAIN, NEW_DOMAIN, 1)
    return uri

updated_count = 0

# --- Find one project owned by the user ---
project = projects.find_one({
    "isArchived": { "$ne": True },
    "ownerId": TEST_USER_ID,
    "name": TEST_PROJECT_NAME
})

if project:
    update_needed = False
    updates = {}

    # --- Main repo ---
    main_uri = project.get("mainRepository", {}).get("uri")
    new_main_uri = replace_domain(main_uri)
    if main_uri != new_main_uri:
        updates["mainRepository.uri"] = new_main_uri
        update_needed = True

    # --- Imported repos ---
    original_repos = project.get("repositories", [])
    new_repos = []
    for repo in original_repos:
        new_repo = repo.copy()
        if "uri" in repo:
            updated_uri = replace_domain(repo["uri"])
            if updated_uri != repo["uri"]:
                new_repo["uri"] = updated_uri
                update_needed = True
        new_repos.append(new_repo)

    if update_needed:
        updates["repositories"] = new_repos
        result = projects.update_one({ "_id": project["_id"] }, { "$set": updates })
        print(f" Updated project '{project['name']}' with changes: {updates}")
        updated_count += result.modified_count
    else:
        print(f"No changes needed for project '{project['name']}'")
else:
    print("No matching project found.")

print(f"\n Test complete. {updated_count} project(s) updated.")

 Updated project 'WineQualityProject' with changes: {'mainRepository.uri': 'https://github.enterprise.com/ddl-tjtolulope/wine-quality-workshop.git', 'repositories': []}

 Test complete. 1 project(s) updated.


In [20]:
import os
from pymongo import MongoClient
from bson import ObjectId

# --- Config ---
MONGO_HOST_TEMPLATE = "mongodb://mongodb-replicaset.{}.svc.cluster.local:27017"
MONGO_DB_NAME = "domino"
MONGO_COLLECTION = "projects"

TEST_USER_ID = ObjectId("6841f10349b4c497b102e30b")
TEST_PROJECT_NAME = "test-github-change"

# Replace with real Git repo URIs
FROM_URI = "https://github.com/ddl-tjtolulope/testgithub.git"
TO_URI = "https://github.com/toluawojana/testgithub.git"

# --- Connection ---
platform_namespace = os.environ.get("DOMINO_PLATFORM_NAMESPACE", "domino-platform")
mongo_uri = MONGO_HOST_TEMPLATE.format(platform_namespace)

client = MongoClient(
    mongo_uri,
    username="admin",
    password="fL9SUiN0ClYzEfKEi1MolvXyM3IaMxoE",
    authSource="admin",
    authMechanism="SCRAM-SHA-1",
)

db = client[MONGO_DB_NAME]
projects = db[MONGO_COLLECTION]

updated_count = 0

# --- Find the specific project ---
project = projects.find_one({
    "isArchived": { "$ne": True },
    "ownerId": TEST_USER_ID,
    "name": TEST_PROJECT_NAME
})

if project:
    update_needed = False
    updates = {}

    print(f"\n🔍 Checking project: {project['name']}")

    # --- Update mainRepository.uri if it matches FROM_URI ---
    main_uri = project.get("mainRepository", {}).get("uri")
    if main_uri == FROM_URI:
        updates["mainRepository.uri"] = TO_URI
        update_needed = True
        print(f"🔧 mainRepository.uri updated:\n   OLD: {FROM_URI}\n   NEW: {TO_URI}")

    # --- Update any imported repositories if they match FROM_URI ---
    updated_repos = []
    for repo in project.get("repositories", []):
        new_repo = repo.copy()
        if repo.get("uri") == FROM_URI:
            new_repo["uri"] = TO_URI
            update_needed = True
            print(f"🔧 importedRepository updated:\n   OLD: {FROM_URI}\n   NEW: {TO_URI}")
        updated_repos.append(new_repo)

    if update_needed:
        updates["repositories"] = updated_repos
        projects.update_one({ "_id": project["_id"] }, { "$set": updates })
        updated_count += 1
        print(f"\n✅ Successfully updated project '{project['name']}'")
    else:
        print(f"ℹ️ No matching URIs found in '{project['name']}' to update.")
else:
    print("❌ No matching project found.")

print(f"\n📦 Test complete. {updated_count} project(s) updated.")


🔍 Checking project: test-github-change
🔧 mainRepository.uri updated:
   OLD: https://github.com/ddl-tjtolulope/testgithub.git
   NEW: https://github.com/toluawojana/testgithub.git

✅ Successfully updated project 'test-github-change'

📦 Test complete. 1 project(s) updated.
